In [1]:
#   https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/structured-outputs?tabs=python


In [1]:
import openai
import os
import configparser

import dotenv
dotenv.load_dotenv()


True

In [6]:
from openai import AzureOpenAI

client = AzureOpenAI(
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
  api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
  api_version="2024-05-01-preview"
)

model = os.getenv("AZURE_OPENAI_MODEL")


In [7]:
json_schema = {
  "name": "customer_support_response",
  "schema": {
    "type": "object",
    "properties": {
      "responseText": {
        "type": "string"
      },
      "intent": {
        "type": "string"
      },
      "confidenceScore": {
        "type": "number"
      },
      "timestamp": {
        "type": "string",
        "format": "date-time"
      }
    },
    "required": [
      "responseText",
      "intent",
      "confidenceScore",
      "timestamp"
    ]
  }
}

In [8]:
print(model)

gpt-4o-2024-08-06


In [9]:

response = client.chat.completions.create(
    model=model, # replace with the model deployment name of your o1-preview, or o1-mini model
    messages=[
        {"role": "user", "content": "What steps should I think about when writing my first Python API?"},
    ],
    max_completion_tokens = 5000

)

print(response.model_dump_json(indent=2))

{
  "id": "chatcmpl-Awkdj30xVffPZwT4NkLedvBo1h2UP",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "When writing your first Python API, it's important to follow a structured approach to ensure that your API is efficient, scalable, and maintainable. Here are the steps you should consider:\n\n1. **Define Your API Requirements:**\n   - Determine the purpose and functionality of your API.\n   - Identify the resources and endpoints you need.\n   - Decide on the data format (typically JSON or XML).\n\n2. **Design Your API:**\n   - Plan out your API routes and endpoints.\n   - Consider RESTful principles for design, focusing on stateless operations.\n   - Define request methods (GET, POST, PUT, DELETE) for each endpoint.\n   - Specify input and output structures including data validation rules.\n\n3. **Choose a Framework:**\n   - Select a Python web framework like Flask, Django, or FastAPI based on your needs

In [15]:
from pydantic import BaseModel
from typing import List

class MedicalCoding(BaseModel):
    id: int
    code: str
    desc: str

class MedicalCodingList(BaseModel):
    codes: List[MedicalCoding]


completion = client.beta.chat.completions.parse(
    model=model, 
    messages=[
        {"role": "system", "content": "Extract the medical code information."},
        {"role": "user", "content": "some text with informaiton about medical code 1234 with id 5678 and description of the code as fracture, code 1122 with id 442322 description inflammation"},
    ],
    response_format=MedicalCodingList,
)

event = completion.choices[0].message.parsed

print(event)
print(completion.model_dump_json(indent=2))

codes=[MedicalCoding(id=5678, code='1234', desc='fracture'), MedicalCoding(id=442322, code='1122', desc='inflammation')]
{
  "id": "chatcmpl-AwkqLbqvfUxkLAIP3mU7YzMivUJgf",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "{\"codes\":[{\"id\":5678,\"code\":\"1234\",\"desc\":\"fracture\"},{\"id\":442322,\"code\":\"1122\",\"desc\":\"inflammation\"}]}",
        "refusal": null,
        "role": "assistant",
        "function_call": null,
        "tool_calls": [],
        "parsed": {
          "codes": [
            {
              "id": 5678,
              "code": "1234",
              "desc": "fracture"
            },
            {
              "id": 442322,
              "code": "1122",
              "desc": "inflammation"
            }
          ]
        }
      },
      "content_filter_results": {
        "hate": {
          "filtered": false,
          "severity": "safe"
        },
        "self_harm"

In [16]:
for code in event.codes:
    print(f"ID: {code.id}, Code: {code.code}, Description: {code.desc}")

ID: 5678, Code: 1234, Description: fracture
ID: 442322, Code: 1122, Description: inflammation
